# LECTURA DE CORREOS DE GMAIL

## CREAR CLIENTE DEL MAIL

In [9]:
# Python 3.8.0
import time
import imaplib
import email
import csv
import traceback
import re
import webbrowser
import os
import unidecode
import unicodedata
import datetime as dt
from datetime import timezone
from email.header import Header
from email.header import decode_header
from unicodedata import normalize
from sklearn import preprocessing
from nltk.tokenize import RegexpTokenizer  
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

from nltp import Preprocessor
from warnings import simplefilter
from nltk.tokenize import RegexpTokenizer  
simplefilter(action='ignore', category=FutureWarning)

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# -------------------------------------------------
#
# Utility to read email from Gmail Using Python
#
# ------------------------------------------------
# account credentials
username = "a.vazquezgomez1996@gmail.com"
with open("contraseña.txt", "r") as fd:
        password = fd.read().strip()

In [10]:
SMTP_SERVER = "imap.gmail.com" 
SMTP_PORT = 993
mail = imaplib.IMAP4_SSL(SMTP_SERVER)
mail.login(username,password)
mail.select('inbox')

('OK', [b'9'])

In [11]:
data = mail.search(None, 'ALL')
mail_ids = data[1]
id_list = mail_ids[0].split()   
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])
url = ''

### LEER Y ALMACENAR CORREOS DEL GMAIL

In [12]:
def get_body(msg):
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else:
        return msg.get_payload(None, True)

In [13]:
gmails_file = open("gmail_mails.csv", "w+", newline = '')
writer = csv.writer(gmails_file)
writer.writerow(["from", "subject", "body", "urls"])

24

In [14]:
def decode_mime_words(s):
    return u''.join(
        word.decode(encoding or 'utf8') if isinstance(word, bytes) else word
        for word, encoding in email.header.decode_header(s))

In [15]:
for i in range(latest_email_id,first_email_id-1, -1):
    data = mail.fetch(str(i), '(RFC822)' )
    for response_part in data:
        arr = response_part[0]
        if isinstance(arr, tuple):
            msg = email.message_from_string(str(arr[1],'utf-8'))
            email_subject_str = decode_mime_words(msg['subject'])
            email_subject_accent = str.join(" ", email_subject_str.splitlines())
            email_subject = unidecode.unidecode(email_subject_accent)
            email_from = msg['from']
            email_body_str = str(get_body(msg).decode('utf-8'))
            email_body_accent = str.join(" ", email_body_str.splitlines())
            email_body = unidecode.unidecode(email_body_accent)
            urls_raw = re.search("(?P<url>https?://[^\s]+)", email_body)
            if urls_raw is not None:
                urls = re.search("(?P<url>https?://[^\s]+)", email_body).group("url")
                print(urls)
            else:
                urls = ''
            writer.writerow([email_from, email_subject, email_body, urls])
            
gmails_file.close()

https://store.asuswebstorage.com/special-offer/?utm_source=167&utm_medium=button&utm_campaign=SystemMail&coupon1=M167500G300&coupon2=M1671TB450>
http://www.iptruster.com/lt/index.php/campaigns/jl459v45fkc36/track-url/vd76640nlg51e/4480a6c0f44350ff8cd9a942cca4bd144c44bf8f>
https://www.google.com/url?q=http://www.phxaffiliates.com/&amp;source=gmail&amp;ust=1623389582570000&amp;usg=AFQjCNFWg9FxdFzAL_xU3LHI8FC4ytV5zA"
http://code.google.com/a/apache-extras.org/p/phpmailer/)
https://bit.ly/3pBFs2i


# CLASIFICACIÓN DE CORREOS DE GMAIL

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

### PREPROCESAMIENTO DE DATOS

In [ ]:
gmails = pd.read_csv('gmail_mails.csv')
gmails

In [ ]:
# Data Cleaning for Mails
gmails_clean = []
lemmatizer = WordNetLemmatizer()

for gmail in gmails['body']:
    # Remove tabs and new lines
    removed_tabs_newline = re.sub('[\n|\t]',' ', gmail)
    
    # Remove special characters and digits
    removed_spchar_digits = re.sub('[^a-zA-Z]',' ',removed_tabs_newline)
    
    # Convert emails into lower case
    lower_case_gmail = removed_spchar_digits.lower()
    
    # Tokenize the emails by words / split by words
    tokenized_gmail = lower_case_gmail.split()
    
    # Remove stopwords
    filtered_words = [word for word in tokenized_gmail if word not in stopwords.words('english')]
    
    # Lemmetize words
    lemmetized_words = [lemmatizer.lemmatize(word) for word in tokenized_gmail]
    
    # Build corpus of emails
    gmail = ' '.join(lemmetized_words)
    gmails_clean.append(gmail)

In [ ]:
# Creating vectors using TF-IDF

tfidf = TfidfVectorizer(max_features=5000)
vectors = tfidf.fit_transform(gmails_clean).toarray()

# Extracting independent and dependent variables from the dataset
X_mails = pd.DataFrame(vectors)
print(X_mails)
print(X_mails.shape)

In [ ]:
gmails_urls = gmails
gmails_urls = gmails_urls.drop(columns=['from', 'subject', 'body'])
gmails_urls.dropna(inplace=True)
print(gmails_urls)

In [ ]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')
gmails_urls.loc[:,'text_tokenized'] = gmails_urls.urls.map(lambda t: tokenizer.tokenize(t))
gmails_urls.head()

In [ ]:
root_words = SnowballStemmer("english")
gmails_urls.loc[:,'root_words'] = gmails_urls['text_tokenized'].map(lambda l: [root_words.stem(word) for word in l])
gmails_urls.head()

In [ ]:
gmails_urls.loc[:,'text_sent'] = gmails_urls['root_words'].map(lambda l: ' '.join(l))
gmails_urls.head()

In [ ]:
c = CountVectorizer()
X_urls = c.fit_transform(gmails_urls.text_sent)
print(X_urls)

### PREDICCIONES CON CORREOS LEIDOS

In [ ]:
gmails_predict = gmails
gmails_predict

In [ ]:
# load model from file
import pickle
loaded_model_mails = pickle.load(open("NaiveBayesClassifier_mails.pickle.dat", "rb"))
loaded_model_mails

In [ ]:
# load model from file
loaded_model_urls = pickle.load(open("NaiveBayesClassifier_urls.pickle.dat", "rb"))
loaded_model_urls

In [ ]:
ham_gmails = open('ham_gmails.txt', 'w+')
spam_gmails = open('spam_gmails.txt', 'w+')
phishing_gmails = open('phishing_gmails.txt', 'w+')

In [ ]:
url_predict = loaded_model_urls.predict(X_urls)
mails_urls['url label'] = url_predict
print(url_predict)

In [ ]:
gmail_predict = loaded_model_mails.predict(X_mails)
gmails_predict['mail label'] = gmail_predict
gmail_predict.shape()

In [ ]:
for i, rows in gmails_predict.iterrows():
    if gmail_predict['urls'][i] is not None:
        if gmail_predict['url label'][i] is 1:
            phishing_gmails.write("From : " + gmail_predict['from'][i]+ '\n')
            phishing_gmails.write("Subject : " + gmail_predict['subject'][i]+ '\n')
            phishing_gmails.write(gmail_predict['content'][i]+ '\n')
            break
    if gmail_predict['mail label'][i] is 1:
        spam_gmails.write("From : " + gmail_predict['from'][i]+ '\n')
        spam_gmails.write("Subject : " + gmail_predict['subject'][i]+ '\n')
        spam_gmails.write(gmail_predict['content'][i]+ '\n')
    else:
        ham_gmails.write("From : " + gmail_predict['from'][i]+ '\n')
        ham_gmails.write("Subject : " + gmail_predict['subject'][i]+ '\n')
        ham_gmails.write(gmail_predict['content'][i]+ '\n')
        
phishing_gmails.close()
spam_gmails.close()
ham_gmails.close()
gmails.close()
gmail_predict.close()